In [135]:
### Import standard functions ###
import os, time, csv, optparse
import numpy as np
from pylab import *
import json
import pandas as pd

In [211]:
# Define set of parsing functions
def CustomParser(data):
    j1 = json.loads(data)
    return j1

def filter_json(x):
    x=x[0]
    try:
        x['value']=x['value'][0]
    except:
        x['value'] = {u'answers': {}, u'choice': {}, u'filters': {}}
    return x

def extract_choice(x):
    y = []
    y.append(str(x['value']['choice']))
    return y

def extract_tasks(x):
    x=x['task']
    return x

def extract_answers(x):
    x=x['value']['answers']
    return x

def extract_filters(x):
    x=['value']['filters']
    return x

def extract_zooID(x):
    x=x.keys()[0]
    return x

def extract_FileName1(x):
    try:
        x = x[x.keys()[0]]['Filename1'].split('_')[1]
    except:
        x = []
    return x

def check_upload(x):
    if len(x.split(';')) == 4:
        x = True
    else:
        x = False
    return x

def check_anno(x):
    if len(x) == 1:
        x = True
    else:
        x = False
    return x

def convert_to_int(x):
    try:
        x=int(x)
    except:
        x=False
    return x

In [212]:
#opts = parse_commandline()
class_file = "gravity-spy-classifications.csv"

In [286]:
# Reading in csv with custom read for those column in JSON format
data1 = pd.read_csv(class_file,converters={'annotations':CustomParser,'subject_data':CustomParser})

# change ID to int
data1['user_id'] = data1['user_id'].apply(convert_to_int)

# Doing a mild work around for the json format of the annontation column
data1['annotations']    = data1['annotations'].apply(filter_json)
# Extracting choice and making it a column
data1['choice']         = data1['annotations'].apply(extract_choice)
# Extracting the task entry and making it a column
data1['tasks']          = data1['annotations'].apply(extract_tasks)
# Extracting answers and making it a column
data1['answers']        = data1['annotations'].apply(extract_answers)
# Extracting zooniverse ID it gave this subject and making it a column
data1['zooID']          = data1['subject_data'].apply(extract_zooID) 
# Extracting uniqueID assigned to the image during image creation and making it a column
data1['imageID']       = data1['subject_data'].apply(extract_FileName1)
# Making sure that the subject_ids for a given classification is 4. If not I uploaded the images wrong
# for that subject
data1['goodUpload']     = data1['subject_ids'].apply(check_upload)
# Making sure the number of annotation is of size 1 (i.e. they did not do multiple annotation)
data1['numAnnotations'] = data1['choice'].apply(check_anno)

#dropping annotations,subject_data, and subject_ids
data1 = data1.drop('annotations',1)
data1 = data1.drop('subject_data',1)
data1 = data1.drop('subject_ids',1)

In [287]:
# Workflow Versions we will work with
versions = [36.7,380.7,692.102,714.11399999999992]
data1['goodWorkFlow'] = (data1['workflow_version'].isin(versions))

In [297]:
# Adding a column to indicate what # annontation this is for a particular user
data1 = data1.sort_values(['user_name','classification_id'])

numOfClassification = []
iN = 1
for iT in data1['user_name'].duplicated():
    if not iT:
        iN = 1
    else:
        iN=iN +1
    numOfClassification.append(iN)
    
data1['numClassification'] = numOfClassification

In [300]:
data1 = data1.sort_index()
data1

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,choice,tasks,answers,zooID,imageID,goodUpload,numAnnotations,goodWorkFlow,numClassification
0,9525037,lmaesampson,1414392,5f156a8c2b8d034ee3ad,1479,Apprentice - Jump in here for advanced Glitch ...,20.300,2016-01-28 19:43:53 UTC,NaN,NaN,"{""session"":""11eb1a69d6e760c329984c2e8315e0b93d...",[{}],T1,{},1075876,Lqw3QB36UT,True,True,False,1
1,9604193,crowston,336603,dbc82e33a3b04ead00b1,1479,Apprentice - Jump in here for advanced Glitch ...,44.600,2016-02-02 16:10:39 UTC,NaN,NaN,"{""session"":""75c5ec8ed51acbe57ebbf4832889138f12...",[BLP],T1,{},1076285,HTlgwFjWZn,True,True,False,1
2,9604196,crowston,336603,b520b59e40070c7f7b20,1479,Apprentice - Jump in here for advanced Glitch ...,44.600,2016-02-02 16:10:50 UTC,NaN,NaN,"{""session"":""75c5ec8ed51acbe57ebbf4832889138f12...",[60HZPWRMNS],T1,{},1077316,suKWBhith1,True,True,False,2
3,9604454,crowston,336603,b520b59e40070c7f7b20,1479,Apprentice - Jump in here for advanced Glitch ...,44.600,2016-02-02 16:21:35 UTC,NaN,NaN,"{""session"":""cb966ac7ba0eaf72b90dd9b3a38a850bf9...",[BLP],T1,{},1076889,6rkFgYSBKl,True,True,False,3
4,9604601,crowston,336603,b520b59e40070c7f7b20,1479,Apprentice - Jump in here for advanced Glitch ...,44.600,2016-02-02 16:26:48 UTC,NaN,NaN,"{""session"":""cb966ac7ba0eaf72b90dd9b3a38a850bf9...",[BLP],T1,{},1079043,IbPP81GsPI,True,True,False,4
5,9604606,crowston,336603,b520b59e40070c7f7b20,1479,Apprentice - Jump in here for advanced Glitch ...,44.600,2016-02-02 16:26:59 UTC,NaN,NaN,"{""session"":""cb966ac7ba0eaf72b90dd9b3a38a850bf9...",[NNFTHBV],T1,{},1077431,q9qihm2DAZ,True,True,False,5
6,9604614,crowston,336603,dbc82e33a3b04ead00b1,1479,Apprentice - Jump in here for advanced Glitch ...,44.600,2016-02-02 16:27:14 UTC,NaN,NaN,"{""session"":""cb966ac7ba0eaf72b90dd9b3a38a850bf9...",[SCTTRDLGHT],T1,{},1078615,wIKCHx7qng,True,True,False,6
7,9625530,costerlusyr.edu,1303502,dbc82e33a3b04ead00b1,1479,Apprentice - Jump in here for advanced Glitch ...,44.600,2016-02-03 16:57:34 UTC,NaN,NaN,"{""session"":""7b60c2f156e7ae1a05fe243bcea06c82ba...",[HRMNCS],T1,{},1077431,q9qihm2DAZ,True,True,False,1
8,9625533,costerlusyr.edu,1303502,dbc82e33a3b04ead00b1,1479,Apprentice - Jump in here for advanced Glitch ...,44.600,2016-02-03 16:57:44 UTC,NaN,NaN,"{""session"":""7b60c2f156e7ae1a05fe243bcea06c82ba...",[SCTTRDLGHT],T1,{},1075876,Lqw3QB36UT,True,True,False,2
9,10295716,mzevin1,796717,e0e08dbd7223d98f05c1,1479,Apprentice - Jump in here for advanced Glitch ...,77.360,2016-03-04 16:40:18 UTC,NaN,NaN,"{""session"":""65e4559affff1d4e2f0592b12c50a37aaf...",[NNFTHBV],T1,{},1688528,[],False,True,False,1


In [195]:
for iV in versions:
    version1 = np.unique(data1[data1['workflow_version'] == iV]['choice'])
    print("version {0}".format(iV))
    print("length {0}".format(len(data1[data1['workflow_version'] == iV])))
    print(version1)
    print("end")
    

version 36.7
length 8058
[['BLP'] ['CHRP'] ['KFSH'] ['PWRLN60HZ'] ['SCTTRDLGHT'] ['WHSTL']]
end
version 380.7
length 29132
[['45MHZLGHTMDLTN'] ['BLP'] ['CHRP'] ['CLBRTNLN300HZ'] ['DNTSGLTCH']
 ['KFSH'] ['LWFRQNCBRST'] ['NNFTHBV'] ['PWRLN60HZ'] ['SCRTCH']
 ['SCTTRDLGHT'] ['VLNHRMNC500HZ'] ['WHSTL'] ['WNDRNGLN'] ['XTRMLLD']]
end
version 692.102
length 11874
[['45MHZLGHTMDLTN'] ['50HZ'] ['BLP'] ['CHRP'] ['HLX'] ['KFSH']
 ['LWFRQNCBRST'] ['LWFRQNCLN'] ['NGLTCH'] ['NNFTHBV'] ['PRDDVS']
 ['PWRLN60HZ'] ['RPTNGBLPS'] ['SCRTCH'] ['SCTTRDLGHT'] ['TMT']
 ['VLNMDHRMNC500HZ'] ['WHSTL'] ['WNDRNGLN'] ['XTRMLLD']]
end
version 714.114
length 6373
[['BLP'] ['CHRP'] ['HLX'] ['KFSH'] ['LGHTMDLTN'] ['LWFRQNCBRST']
 ['LWFRQNCLN'] ['NGLTCH'] ['NNFTHBV'] ['PRDDVS'] ['PWRLN60HZ']
 ['RCMPRSSR50HZ'] ['RPTNGBLPS'] ['SCRTCH'] ['SCTTRDLGHT'] ['TMT']
 ['VLNMDHRMNC500HZ'] ['WHSTL'] ['WNDRNGLN'] ['XTRMLLD']]
end


In [109]:
### DICT FOR CONVERTING LABELS TO INTEGERS ###
label_dict = {'45MHZLGHTMDLTN':1,'LGHTMDLTN':1,'50HZ':2,'RCMPRSSR50HZ':2,'BLP':3,'CHRP':4,'XTRMLLD':5,'HLX':6,'KFSH':7,
              'LWFRQNCBRST':8,'LWFRQNCLN':9,'NGLTCH':10,'DNTSGLTCH':10,'NNFTHBV':11,'PRDDVS':12,'60HZPWRLN':13,'60HZPWRMNS':13,
              'PWRLN60HZ':13,'RPTNGBLPS':14,'SCTTRDLGHT':15,'SCRTCH':16,'TMT':17,'VLNHRMNC500HZ':18,'VLNMDHRMNC500HZ':18,
              'HRMNCS':18,'WNDRNGLN':19,'WHSTL':20}

In [5]:
### READ FILE FOR CONVERTING imageIDs ###
id_match = pd.read_csv('IDmatchall.txt') #read in file
tmp_new = []
tmp_old = []
no_match = [] #list of imageIDs with no match (timestamp errors)
id_dict = {} #empty dict for converting imageIDs

for i in id_match['# New       Old']:
    if len(i)>10:
        i = i.split(' ')
        tmp_new.append(i[0])
        tmp_old.append(i[1])

id_match = pd.DataFrame({'new':tmp_new,'old':tmp_old})

for a,b in zip(id_match['new'],id_match['old']):
    id_dict[b] = a

In [30]:
### APPEND ALL PERTINENT DATA TO LIST ###

tmp_user= []
tmp_user_id = []
tmp_workflow = []
tmp_task = []
tmp_choice = []
tmp_retired = []
tmp_unique_id = []
tmp_zoo_id = []

for i in range(len(data)):
    # create list to hold to output information of each classification
    output = []
    # Check that there was only 1 choice made...
    annotations = json.loads(data[i][11])
    idcheck = data[i][2]
    if str(annotations).count('choice') == 1 and not np.isnan(idcheck):
        user = data[i,1]
        user_id = data[i,2]
        workflow = data[i,5]
        
        # annotations
        task = annotations[0]["task"]
        choice = annotations[0]["value"][0]["choice"]
        
        # subject data
        subject_data = json.loads(data[i][12])
        for key in subject_data:
            zoo_id = key
            retired = subject_data[key]['retired']
            unique_id = subject_data[key]['subject_id']
        
        # Append this information into a temporary output file
        tmp_user_id.append(user_id)
        tmp_workflow.append(workflow)
        tmp_task.append(task)
        tmp_choice.append(choice)
        tmp_retired.append(retired)
        tmp_unique_id.append(unique_id)
        tmp_zoo_id.append(zoo_id)
        
# Store each of the classification data
classifications = pd.DataFrame({'imageID':tmp_unique_id,'userID':tmp_user_id,'workflow':tmp_workflow,
                                'task':tmp_task,'label':tmp_choice,'type':tmp_retired, 'zooID':tmp_zoo_id})

In [127]:
images

,ML_posterior,imageID,labels,truelabel,type,userIDs,zooID
0,[],HauRDnEd8q,"[10, 10, 10, 10, 10, 10, 9, 10]",-1,T,"[835158, 1498433, 637439, 239792, 132, 82, 227...",2210216
1,[],qOOEUMNyGi,[7],-1,T,[239792],2207683
2,[],GGfpUmYtJD,"[9, 9, 9, 9, 9]",-1,T,"[1498519, 1498701, 530281, 239792, 243841]",2209323
3,[],N70PSqtshJ,[11],-1,T,[322252],2200834
4,[],j99whZS7hL,"[9, 15, 15, 15]",-1,T,"[1047240, 239792, 82, 497320]",2211647
5,[],tRBVjF55Ah,"[11, 8, 8, 11, 8]",-1,T,"[1498519, 239792, 82, 322252, 2274]",2219760
6,[],NCCpMZbVi0,[9],-1,T,[102001],2215367
7,[],EPTjErvJJi,"[15, 13, 13, 11]",-1,T,"[835158, 239792, 2909, 82]",2209896
8,[],FBQL7ephxf,"[9, 9, 9, 9, 9]",-1,T,"[1498519, 1498433, 239792, 82, 322252]",2196270
9,[],RFhohs0FxA,"[13, 13, 13, 11]",-1,T,"[82, 239792, 2274, 322252]",2215595


In [85]:
uniques = set(np.unique(classifications['imageID'])) #create set of unique imageIDs
keys = set(id_dict.keys()) #create set of new imageIDs from id_dict
uniques = list(uniques.intersection(keys)) #find intersection of sets, convert to list

In [24]:
#function to create lists of empty lists
def emptylist(x):
    elist = []
    for i in range(x):
        elist.append([])
    return elist

In [120]:
### READ CLASSIFICATIONS FROM GRAVSPY BETA ###

#turn off unnecessary warning about setting values to slice of dataframe
pd.options.mode.chained_assignment = None  # default='warn'

#create dataframe, length of uniques, without labels or userIDs
images = pd.DataFrame({'type':['T']*len(uniques),'labels':emptylist(len(uniques)),
                        'userIDs':emptylist(len(uniques)),'ML_posterior':emptylist(len(uniques)),
                        'truelabel':[-1]*len(uniques),'imageID':uniques,'zooID':emptylist(len(uniques))})

for i in range(len(uniques)): #iterate over unique imageIDs
    
    classifications_idx = np.where((uniques[i] == classifications['imageID']))[0][0]
    
    images['zooID'][i] = int(classifications.loc[[classifications_idx], 'zooID'])
    
    for locations in np.where(uniques[i] == classifications['imageID']): #iterate over arrays of where unique imageID appears
        
        images_idx = np.where(uniques[i] == images['imageID'])[0][0] #find index of line in images where unique imageID appears
        
        for location in locations: #iterate over elements in array of locations in classifications where unique imageID appears
        
            images['labels'][images_idx].append(label_dict[classifications['label'][location]]) #append numeric label
            images['userIDs'][images_idx].append(int(classifications['userID'][location])) #append userID
            
for imageID in images['imageID']:

    imageID = id_dict[imageID]

In [133]:
### READ CLASSIFICATIONS OF GOLDEN IMAGES ###

goldendata = pd.read_csv('GLabel.csv')

for i in range(len(goldendata)): #iterate over data
    
    try:
        images_idx = np.where(int(goldendata['zooID'][i]) == images['zooID'])[0][0] #find location in images dataframe
        images['truelabel'][images_idx] = int(goldendata['Classification'][i]) #change true label to golden classification
        images['type'][images_idx] = 'G' #change image type to golden
        
    except:
        pass #to catch errors caused by images in goldendata not being in images dataframe

In [134]:
images

,ML_posterior,imageID,labels,truelabel,type,userIDs,zooID
0,[],HauRDnEd8q,"[10, 10, 10, 10, 10, 10, 9, 10]",-1,T,"[835158, 1498433, 637439, 239792, 132, 82, 227...",2210216
1,[],qOOEUMNyGi,[7],-1,T,[239792],2207683
2,[],GGfpUmYtJD,"[9, 9, 9, 9, 9]",-1,T,"[1498519, 1498701, 530281, 239792, 243841]",2209323
3,[],N70PSqtshJ,[11],-1,T,[322252],2200834
4,[],j99whZS7hL,"[9, 15, 15, 15]",-1,T,"[1047240, 239792, 82, 497320]",2211647
5,[],tRBVjF55Ah,"[11, 8, 8, 11, 8]",-1,T,"[1498519, 239792, 82, 322252, 2274]",2219760
6,[],NCCpMZbVi0,[9],-1,T,[102001],2215367
7,[],EPTjErvJJi,"[15, 13, 13, 11]",-1,T,"[835158, 239792, 2909, 82]",2209896
8,[],FBQL7ephxf,"[9, 9, 9, 9, 9]",-1,T,"[1498519, 1498433, 239792, 82, 322252]",2196270
9,[],RFhohs0FxA,"[13, 13, 13, 11]",-1,T,"[82, 239792, 2274, 322252]",2215595
